### Simple Jupyter Chess 01
Adaptation building upon:
 - https://jupyter.brynmawr.edu/services/public/dblank/CS371%20Cognitive%20Science/2016-Fall/Programming%20a%20Chess%20Player.ipynb
 
User can play white or black vs. a simple chess engine that selects at random from available legal moves.
Or, the program can be set to play itself.  Use `re-run the whole notebook` or step through in order.  
Select playing option, then play.
 
Jopin Talgwin
2022-02-08

#### Imports

In [ ]:
import random
import time

import chess
import chess.svg
import chess.pgn

from IPython.display import display, clear_output

#### Constants

In [ ]:
PLAY_AS_WHITE = "w"
PLAY_AS_BLACK = "b"
AUTO_PLAY = "a"

PLAY_OPTIONS = [
    PLAY_AS_WHITE, 
    PLAY_AS_BLACK,
    AUTO_PLAY,
]

AUTO_PLAY_WAIT = 1                # Duration in sec. of each board display during autoplay.
DISPLAY_WAIT = 0.2                # Needed to get the display working for me.

#### Function Definitions

In [ ]:
def random_move(board):
    move = random.choice(list(board.legal_moves))
    return move.uci()

def who(player):
    return "White" if player == chess.WHITE else "Black"

def svg_chess(board):
    """
    Uses the python-chess `chess.svg.board()` function to return a renering of 
    `board` as a svg impage oriented from the current board.turn POV for use 
    in `IPython.display.display();`).      
    """
    return chess.svg.board(
        board, 
        orientation=board.turn,
        size=400,
    ) 

def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

def user_move(board):
    display(svg_chess(board))
    uci = get_move("%s's move [q to quit]> " % who(board.turn))
    legal_uci_moves = [move.uci() for move in board.legal_moves]
    while uci not in legal_uci_moves:
        print("Legal moves: " + (",".join(sorted(legal_uci_moves))))
        uci = get_move("%s's move[q to quit]> " % who(board.turn))
    return uci

def choose_play_options():
    play_option = None
    while play_option not in PLAY_OPTIONS:
        print("You can choose to play White or Black, or have the computer play both sides")
        play_option = input(", ".join(PLAY_OPTIONS)+":").lower()
    return play_option

#### Select Playing Option

In [ ]:
play_option = choose_play_options()

#### Game Play

In [ ]:
# Instantiate a python-chess chess.Board().
board = chess.Board()                            

# Note the starting time for use later.
yr = str(time.localtime(time.time()).tm_year)

mo = str(time.localtime(time.time()).tm_mon)
if len(mo) == 1:
    mo = "0"+mo

day = str(time.localtime(time.time()).tm_mday)
if len(day) == 1:
    day = "0"+day

# Begin game play.
try:
    while not board.is_game_over(claim_draw=True):
        if play_option == AUTO_PLAY:
            board.push_uci(random_move(board))
            if board.ply()%10 == 0:
                display(svg_chess(board))
                print(f"Position after {board.ply()//2} moves.")
                time.sleep(AUTO_PLAY_WAIT)
                clear_output()
        else:                
            clear_output()
            time.sleep(DISPLAY_WAIT)                 # Display hack to get proper display.
            if play_option == PLAY_AS_WHITE:
                if board.turn == chess.WHITE:
                    board.push_uci(user_move(board))
                else:
                    board.push_uci(random_move(board))
            else:
                if board.turn == chess.BLACK:
                    board.push_uci(user_move(board))
                else:
                    board.push_uci(random_move(board))
    display(svg_chess(board))
    if board.is_checkmate():
        print("checkmate: " + who(not board.turn) + " wins!")
    elif board.is_stalemate():
        print("draw: stalemate")
    elif board.is_fivefold_repetition():
        print("draw: 5-fold repetition")
    elif board.is_insufficient_material():
        print("draw: insufficient material")
    elif board.can_claim_draw():
        print("draw: claim")

except KeyboardInterrupt: 
    clear_output(wait=True)
    display(svg_chess(board))
    print("Game interrupted!")


In [ ]:
print("Final Position")
display(board)

In [ ]:
# Convert Game Played to pgn
game = chess.pgn.Game.from_board(board)

game.headers["Event"] = "Simple Jupyter Chess Game"
game.headers["Date"] = yr + "." + mo + "."+ day

if play_option == "w":
    game.headers["White"] = "Player"
    game.headers["Black"] = "Random Legal Moves"
elif play_option == "b":
    game.headers["White"] = "Random Legal Moves"
    game.headers["Black"] = "Player"
else:
    game.headers["White"] = "Random Legal Moves"
    game.headers["Black"] = "Random Legal Moves"
    
print(game)